In [3]:
import numpy as np
from math import *
import os
from os.path import join
import bpy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipyvolume as ipv
from numpy.linalg import norm
import torch
import lie_learn.spaces.S2 as S2

In [3]:
def create_cube_grid():
    x_ = np.linspace(-1., 1., 4)
    y_ = np.linspace(-1., 1., 4)
    z_ = np.linspace(-1., 1., 4)

    x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')

    x = x.flatten()
    y = y.flatten()
    z = z.flatten()
    
    return x, y, z

In [4]:
def create_cube_centers():
    x_ = np.linspace(-2/3, 2/3, 3)
    y_ = np.linspace(-2/3, 2/3, 3)
    z_ = np.linspace(-2/3, 2/3, 3)

    x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')

    x = x.flatten()
    y = y.flatten()
    z = z.flatten()
    
    return x, y, z

In [143]:
x_axis, y_axis, z_axis = create_cube_grid()
x_axis_, y_axis_, z_axis_ = create_cube_centers()

In [156]:
fig = ipv.figure()
ipv.scatter(x_axis, y_axis, z_axis, marker="sphere", color="red")
ipv.scatter(x_axis_, y_axis_, z_axis_, marker="sphere", color="blue")
ipv.show()
ipv.save("plt.html")

AttributeError: module 'ipyvolume' has no attribute 'close'

In [145]:
centers = np.array(list(zip(x_axis_, y_axis_, z_axis_)))

In [146]:
fig = ipv.figure()
R = 2 * 1/3 * np.sqrt(3) * 1/2
for center in centers:
    x = R * np.outer(np.cos(u), np.sin(v)) + center[0]
    y = R * np.outer(np.sin(u), np.sin(v))  + center[1]
    z = R * np.outer(np.ones(np.size(u)), np.cos(v)) + center[2]
    ipv.plot_mesh(x, y, z, u=y, v=x, wireframe=False, color="red")
ipv.show()

In [182]:
def get_grids(b, num_grids, base_radius=1, center=[0, 0, 0], grid_type="Driscoll-Healy"):
    """
    :param b: the number of grids on the sphere
    :param base_radius: the radius of each sphere
    :param grid_type: "Driscoll-Healy"
    :param num_grids: number of grids
    :return: [(radius, tensor([2b, 2b, 3])) * num_grids]
    """

    grids = list()
    radiuses = [round(i, 2) for i in list(np.linspace(0, base_radius, num_grids + 1))[1:]]

    # Each grid has differet radius, the radiuses are distributed uniformly based on number
    for radius in radiuses:

        # theta in shape (2b, 2b), range [0, pi]; phi range [0, 2 * pi]
        theta, phi = S2.meshgrid(b=b, grid_type=grid_type)
        theta = torch.from_numpy(theta)
        phi = torch.from_numpy(phi)

        # x will be reshaped to have one dimension of 1, then can broadcast
        # look this link for more information: https://pytorch.org/docs/stable/notes/broadcasting.html
        x_ = radius * torch.sin(theta) * torch.cos(phi)
        x = x_.reshape((1, 4 * b * b))  # tensor -> [1, 4 * b * b]
        x = x + center[0]

        y_ = radius * torch.sin(theta) * torch.sin(phi)
        y = y_.reshape((1, 4 * b * b))
        y = y + center[1]

        z_ = radius * torch.cos(theta)
        z = z_.reshape((1, 4 * b * b))
        z = z + center[2]

        grid = torch.cat((x, y, z), dim=0)  # -> [3, 4b^2]
        grid = grid.transpose(0, 1)  # -> [4b^2, 3]

        grid = grid.view(2 * b, 2 * b, 3)  # -> [2b, 2b, 3]
        grid = grid.float().cuda()

        grids.append( (radius, grid) )

    assert len(grids) == num_grids
    return grids


def get_cube_grids(b, num_grids, base_radius):
    # Create sphere centers
    x_ = np.linspace(-2/3, 2/3, 3)
    y_ = np.linspace(-2/3, 2/3, 3)
    z_ = np.linspace(-2/3, 2/3, 3)

    x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')

    x = x.flatten()
    y = y.flatten()
    z = z.flatten()
    
    centers = np.array(list(zip(x, y, z)))
    
    # Create grids based on each center
    all_grids = []
    R = 2 * base_radius * 1/3 * np.sqrt(3) * 1/2
    for center in centers:
        grids = [grid for radius, grid in get_grids(b, num_grids, base_radius=R, center=center)]
        grids = torch.stack(grids, dim=0) # tensor -> [num_grids, 2b, 2b, 3]
        all_grids.append(grids)
    all_grids = torch.stack(all_grids, dim=0) # tensor -> [num_centers, num_grids, 2b, 2b, 3]
    return all_grids

In [183]:
grids = get_cube_grids(b=5, num_grids=3, base_radius=1)
print(grids.shape)
grids = grids.transpose(0, 1)
print(grids.shape)

torch.Size([27, 3, 10, 10, 3])
torch.Size([3, 27, 10, 10, 3])


In [27]:
colors = ["#7b0001", "#ff0001", "#ff8db4"]
fig = ipv.figure()
for i, layer in enumerate(grids):
    layer = layer.reshape(-1, 3).transpose(0, 1)
    x_axis = layer[0, :].cpu().numpy()
    y_axis = layer[1, :].cpu().numpy()
    z_axis = layer[2, :].cpu().numpy()
    c = matplotlib.cm.afmhot(np.linspace(0, 1, len(x_axis)))
    ipv.scatter(x_axis, y_axis, z_axis, marker="sphere", color=c)
ipv.show()

In [30]:
c[1000:1005, :]

array([[0.7372549 , 0.2372549 , 0.        , 1.        ],
       [0.7372549 , 0.2372549 , 0.        , 1.        ],
       [0.74509804, 0.24509804, 0.        , 1.        ],
       [0.74509804, 0.24509804, 0.        , 1.        ],
       [0.74509804, 0.24509804, 0.        , 1.        ]])

In [1]:
def rotate_random(deflection=1.0, randnums=None):
    """
    Creates a random rotation matrix.
    
    deflection: the magnitude of the rotation. For 0, no rotation; for 1, competely random
    rotation. Small deflection => small perturbation.
    randnums: 3 random numbers in the range [0, 1]. If `None`, they will be auto-generated.
    """
    # from http://www.realtimerendering.com/resources/GraphicsGems/gemsiii/rand_rotation.c
    
    if randnums is None:
        randnums = np.random.uniform(size=(3,))
        
    theta, phi, z = randnums
    
    theta = theta * 2.0*deflection*np.pi  # Rotation about the pole (Z).
    phi = phi * 2.0*np.pi  # For direction of pole deflection.
    z = z * 2.0*deflection  # For magnitude of pole deflection.
    
    # Compute a vector V used for distributing points over the sphere
    # via the reflection I - V Transpose(V).  This formulation of V
    # will guarantee that if x[1] and x[2] are uniformly distributed,
    # the reflected points will be uniform on the sphere.  Note that V
    # has length sqrt(2) to eliminate the 2 in the Householder matrix.
    
    r = np.sqrt(z)
    Vx, Vy, Vz = V = (
        np.sin(phi) * r,
        np.cos(phi) * r,
        np.sqrt(2.0 - z)
        )
    
    st = np.sin(theta)
    ct = np.cos(theta)
    
    R = np.array(((ct, st, 0), (-st, ct, 0), (0, 0, 1)))
    
    # Construct the rotation matrix  ( V Transpose(V) - I ) R.
    
    M = (np.outer(V, V) - np.eye(3)).dot(R)
    return M


In [4]:
rotate_random()

array([[-0.55074153,  0.40192167, -0.7315345 ],
       [-0.40121719,  0.6410572 ,  0.65427092],
       [ 0.73192112,  0.65383839, -0.19179894]])

In [7]:
def get_spheres(scale=1, subdiv=1, radius=0.562):
    middle_point_cache = {}
    
    def vertex(x, y, z):
        """ Return vertex coordinates fixed to the unit sphere """
        length = np.sqrt(x**2 + y**2 + z**2)
        return [(i * scale) / length for i in (x,y,z)]


    def middle_point(point_1, point_2):
        """ Find a middle point and project to the unit sphere """
        smaller_index = min(point_1, point_2)
        greater_index = max(point_1, point_2)

        key = "{0}-{1}".format(smaller_index, greater_index)

        if key in middle_point_cache:
            return middle_point_cache[key]

        # If it's not in cache, then we can cut it
        vert_1 = verts[point_1]
        vert_2 = verts[point_2]
        middle = [sum(i)/2 for i in zip(vert_1, vert_2)]

        verts.append(vertex(*middle))

        index = len(verts) - 1
        middle_point_cache[key] = index

        return index

    PHI = (1 + np.sqrt(5)) / 2

    verts = [ vertex(-1, PHI, 0), 
         vertex( 1, PHI, 0), 
         vertex(-1, -PHI, 0), 
         vertex( 1, -PHI, 0), 
         vertex(0, -1, PHI), 
         vertex(0, 1, PHI), 
         vertex(0, -1, -PHI), 
         vertex(0, 1, -PHI), 
         vertex( PHI, 0, -1), 
         vertex( PHI, 0, 1), 
         vertex(-PHI, 0, -1), 
         vertex(-PHI, 0, 1), ]

    faces = [ 
        # 5 faces around point 0 
        [0, 11, 5], 
        [0, 5, 1], 
        [0, 1, 7], 
        [0, 7, 10], 
        [0, 10, 11], 
        # Adjacent faces 
        [1, 5, 9], 
        [5, 11, 4], 
        [11, 10, 2], 
        [10, 7, 6], 
        [7, 1, 8], 
        # 5 faces around 3 
        [3, 9, 4], 
        [3, 4, 2], 
        [3, 2, 6], 
        [3, 6, 8], 
        [3, 8, 9], 
        # Adjacent faces 
        [4, 9, 5], 
        [2, 4, 11], 
        [6, 2, 10], 
        [8, 6, 7], 
        [9, 8, 1], 
    ]
    
    # Generate based dots on Sphere surface
    for i in range(subdiv): 
        faces_subdiv = [] 
        for tri in faces: 
            v1 = middle_point(tri[0], tri[1]) 
            v2 = middle_point(tri[1], tri[2]) 
            v3 = middle_point(tri[2], tri[0]) 
            faces_subdiv.append([tri[0], v1, v3]) 
            faces_subdiv.append([tri[1], v2, v1]) 
            faces_subdiv.append([tri[2], v3, v2]) 
            faces_subdiv.append([v1, v2, v3]) 
        faces = faces_subdiv

    def get_radius(scale, verts):
        min_dist = find_min_dist(verts)
        d = 0.5 * min_dist / np.cos(np.pi / 6) 

        x = verts[5]

        v = np.random.rand(3)
        v = v - np.matmul(v, x) * x
        v = v / norm(v) * d

        p = np.cos(norm(v)) * x + np.sin(norm(v)) * v / norm(v)

        origin = np.array([0, 0, 0])
        center = 0.5 * (origin + x)

        radius = euclidean_distance(p, center)

        return radius
    
    # Get embedding sphere centers
    centers = []
    origin = np.array([0, 0, 0])
    for vert in verts:
        center = 0.5 * (origin + vert)
        centers.append(center)
    
    return centers, radius

In [8]:
centers, radius = get_spheres()